## 1. 환경 설정

`(1) LangSmith 설정 확인`
- .env 파일에 아래 내용을 반영
    - LANGCHAIN_TRACING_V2=true  
    - LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"  
    - LANGCHAIN_API_KEY="인증키를 입력하세요"  
    - LANGCHAIN_PROJECT="프로젝트명"  

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

import numpy as np
import pandas as pd

`(3) Env 환경변수`

In [3]:
from dotenv import load_dotenv
load_dotenv()

# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
print("langsmith 추척 여부: ", os.getenv('LANGCHAIN_TRACING_V2'))

langsmith 추척 여부:  None


## 2. Load Data

`(1) Raw Documents`

In [4]:
# 문서를 로드
from langchain_core.documents import Document
import json

final_docs = []

with open('./data/final_docs_ver2.jsonl', 'rb') as f:
    for line in f:
        item = json.loads(line)
        doc = Document(page_content=item['page_content'], metadata=item['metadata'])
        final_docs.append(doc)

print(len(final_docs))

8


`(2) Test Data`

In [4]:
# Test 데이터셋에 대한 QA 생성 결과를 리뷰한 후 다시 로드
import pandas as pd

df_qa_test = pd.read_excel("./data/qa_test_revised.xlsx")

df_qa_test.head()

,context,source,doc_id,question,answer
0,"['.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2...",['data/리비안_KR.txt'],['0'],리비안의 초기 모델은 무엇인가요?,리비안의 초기 모델은 스포츠카 R1입니다.
1,"['.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2...",['data/리비안_KR.txt'],['0'],R1의 좌석 구성은 어떻게 되나요?,R1은 2+2 좌석 구성입니다.
2,"['.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2...",['data/리비안_KR.txt'],['0'],R1은 어떤 구조를 특징으로 하나요?,R1은 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 합니다.
3,"['테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전...",['data/테슬라_KR.txt'],['1'],테슬라는 어디에 본사를 두고 있나요?,테슬라는 텍사스주 오스틴에 본사를 두고 있습니다.
4,"['테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전...",['data/테슬라_KR.txt'],['1'],테슬라는 언제 설립되었나요?,테슬라는 2003년에 설립되었습니다.


`(3) 검색도구 정의`

-  BM25 검색기

In [6]:
# BM25 검색기를 사용하기 위한 준비
from krag.tokenizers import KiwiTokenizer
from krag.retrievers import KiWiBM25RetrieverWithScore

kiwi_tokenizer = KiwiTokenizer(model_type='knlm', typos='basic')

bm25_db = KiWiBM25RetrieverWithScore(
        documents=final_docs, 
        kiwi_tokenizer=kiwi_tokenizer, 
        k=2, 
        threshold=0.0,
    )

# BM25 검색기를 사용하여 문서 검색
query = "테슬라의 회장은 누구인가요?"
retrieved_docs = bm25_db.invoke(query, 2)

# 검색 결과 출력 
print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()

쿼리: 테슬라의 회장은 누구인가요?
검색 결과:
- . 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다

(참고: 이 문서는 테슬라에 대한 정보를 담고 있습니다.) [출처: 테슬라_KR.txt]
----------------------------------------------------------------------------------------------------

- .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다

(참고: 이 문서는 리비안에 대한 정보를 담고 있습니다.) [출처: 리비안_KR.txt]
----------------------------------------------------------------------------------------------------



- Vector Store 로드

In [7]:
# 벡터스토어 로드
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

chroma_db = Chroma(
    embedding_function=embeddings_model,
    collection_name="hf_bge_m3",
    persist_directory="./chroma_db",
)

chroma_k = chroma_db.as_retriever(
    search_kwargs={'k': 2},
)

# 벡터스토어를 사용하여 문서 검색
query = "테슬라의 회장은 누구인가요?"

retrieved_docs = chroma_k.invoke(query)

# 검색 결과 출력
print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()

c:\Users\1\miniconda3\envs\langchain\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
C:\Users\1\AppData\Local\Temp\ipykernel_15412\1459410627.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  chroma_db = Chroma(
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument b

쿼리: 테슬라의 회장은 누구인가요?
검색 결과:
- 테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다

(참고: 이 문서는 테슬라에 대한 정보를 담고 있습니다.) [출처: 테슬라_KR.txt]
----------------------------------------------------------------------------------------------------

- . 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다

(참고: 이 문서는 테슬라에 대한 정보를 담고 있습니다.) [출처: 테슬라_KR.txt]
----------------------------------------------------------------------------------------------------



- Emsemble Hybrid Search 활용

In [8]:
from langchain.retrievers import EnsembleRetriever


# 검색기 초기화 
def create_hybrid_retriever(bm25_db, vector_db, k: int = 4):

    bm25_db.k = k
    chroma_k = vector_db.as_retriever(search_kwargs={'k': k})

    retriever = EnsembleRetriever(
        retrievers=[bm25_db, chroma_k],
        weights=[0.5, 0.5],
    )

    return retriever

hybrid_retriever = create_hybrid_retriever(bm25_db, chroma_db, k=4)

query = "테슬라의 회장은 누구인가요?"
retrieved_docs = hybrid_retriever.invoke(query)

# 검색 결과 출력
print(f"쿼리: {query}")
print("검색 결과:")

for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()

쿼리: 테슬라의 회장은 누구인가요?
검색 결과:
- . 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다

(참고: 이 문서는 테슬라에 대한 정보를 담고 있습니다.) [출처: 테슬라_KR.txt]
----------------------------------------------------------------------------------------------------

- .

2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 테슬라의 전 세계 누적 판매량은 4,962,975대를 초과했습니다. SMT Packaging에 따르면, 2023년 테슬라의 판매량은 전 세계 전기차 시장의 약 12.9%를 차지했습니다

(참고: 이 문서는 테슬라에 대한 정보를 담고 있습니다.) [출처: 테슬라_KR.txt]
----------------------------------------------------------------------------------------------------

- 테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다

(참고: 이 문서는 테슬라에 대한 정보를 담고 있습니다.) [출처: 테슬라_KR.txt]
----------------------------------------------------------------------------------------------------

- .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하

- Cross-Encoder 알고리즘에 기반하여 재정렬 (Re-rank)

In [9]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
re_ranker = CrossEncoderReranker(model=model, top_n=3)

cross_encoder_reranker_retriever = ContextualCompressionRetriever(
    base_compressor=re_ranker, 
    base_retriever=hybrid_retriever,
)

question = "테슬라 회장은 누구인가요?"

retrieved_docs = cross_encoder_reranker_retriever.invoke(question)

print(f"쿼리: {question}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()

쿼리: 테슬라 회장은 누구인가요?
검색 결과:
- 테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다

(참고: 이 문서는 테슬라에 대한 정보를 담고 있습니다.) [출처: 테슬라_KR.txt]
----------------------------------------------------------------------------------------------------

- . 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다

(참고: 이 문서는 테슬라에 대한 정보를 담고 있습니다.) [출처: 테슬라_KR.txt]
----------------------------------------------------------------------------------------------------

- 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다

(참고: 이 문서는 리비안에 대한 정보를 담고 있습니다.) [출처: 리비안_KR.txt]
----------------------------------------------------------------------------------------------------



## 3. LLM 유형 및 답변 생성

### 3-1 RAG Chain

In [18]:
# 각 쿼리에 대한 검색 결과를 한꺼번에 Context로 전달해서 답변을 생성
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

def create_rag_chain(retriever, llm):

    template = """Answer the following question based on this context. If the context is not relevant to the question, just answer with '답변에 필요한 근거를 찾지 못했습니다.'

    [Context]
    {context}

    [Question]
    {question}

    [Answer]
    """

    prompt = ChatPromptTemplate.from_template(template)

    def format_docs(docs):
        return "\n\n".join([f"{doc.page_content}" for doc in docs])

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()} 
        | prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain

In [19]:
# RAG 체인 생성 및 테스트
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

openai_rag_chain = create_rag_chain(cross_encoder_reranker_retriever, llm)

question = "테슬라의 회장은 누구인가요?"

answer = openai_rag_chain.invoke(question)

print(f"쿼리: {question}")
print("답변:")
print(answer)


쿼리: 테슬라의 회장은 누구인가요?
답변:
일론 머스크입니다.


### 3-2 주요 모델 공급자

`(1) Anthropic Claude API`

https://www.anthropic.com/api

In [22]:
# ChatAnthropic - LLM 모델 
from langchain_anthropic import ChatAnthropic

# 모델 로드 
llm = ChatAnthropic(
    model="claude-3-haiku-20240307",
    temperature=0,
    max_tokens=200, 
)

# RAG 체인 생성 및 테스트
anthropic_rag_chain = create_rag_chain(cross_encoder_reranker_retriever, llm)

question = "테슬라의 회장은 누구인가요?"

answer = anthropic_rag_chain.invoke(question)

print(f"쿼리: {question}")
print("답변:")
print(answer)

쿼리: 테슬라의 회장은 누구인가요?
답변:
문서에 따르면 테슬라의 회장은 일론 머스크입니다. 문서에서 "머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다."라고 명시되어 있습니다.


`(2) Google Gemini API`

https://ai.google.dev/

In [23]:
# ChatGoogleGenerativeAI - LLM 모델 
from langchain_google_genai import ChatGoogleGenerativeAI

# 모델 로드 
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# RAG 체인 생성 및 테스트
google_genai_rag_chain = create_rag_chain(cross_encoder_reranker_retriever, llm)

question = "테슬라의 회장은 누구인가요?"

answer = google_genai_rag_chain.invoke(question)

print(f"쿼리: {question}")
print("답변:")
print(answer)


쿼리: 테슬라의 회장은 누구인가요?
답변:
일론 머스크입니다. 



`(3) Ollama - 오픈소스 LLM`

https://ollama.com/

In [24]:
# ChatOllama - LLM 모델 
from langchain_ollama import ChatOllama

# 모델 로드 
llm = ChatOllama(
    model = "llama3.1",
    temperature = 0.8,
    num_predict = 100,
)

# RAG 체인 생성 및 테스트
ollama_rag_chain = create_rag_chain(cross_encoder_reranker_retriever, llm)

question = "테슬라의 회장은 누구인가요?"

answer = ollama_rag_chain.invoke(question)

print(f"쿼리: {question}")
print("답변:")
print(answer)

쿼리: 테슬라의 회장은 누구인가요?
답변:
일론 머스크입니다.


`(4) Groq API - 오픈소스 LLM`

https://groq.com/

In [26]:
# ChatGroq - LLM 모델 
from langchain_groq import ChatGroq

# 모델 로드 
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.0,
    max_tokens=100,
)

# RAG 체인 생성 및 테스트
groq_rag_chain = create_rag_chain(cross_encoder_reranker_retriever, llm)

question = "테슬라의 회장은 누구인가요?"

answer = groq_rag_chain.invoke(question)

print(f"쿼리: {question}")
print("답변:")
print(answer)

쿼리: 테슬라의 회장은 누구인가요?
답변:
일론 머스크


## 4. RAG 답변 평가

### 4-1. Metric Evaluation

`(1) Embedding Distance`
- 임베딩 거리를 사용하여 예측 문자열과 참조 레이블 문자열 간의 의미적 유사성을 측정
- 계산된 거리 점수가 낮을수록 두 문자열의 의미가 더 유사함을 나타내며, 이 방법은 단순 문자열 비교보다 더 풍부한 의미적 평가가 가능

In [ ]:
# LangChain EmbeddingDistanceEvalChain 활용 

from langchain.evaluation import load_evaluator, EvaluatorType, EmbeddingDistance
from langchain_openai import ChatOpenAI

# Evaluator 초기화
embedding_evaluator = load_evaluator(
    evaluator=EvaluatorType.EMBEDDING_DISTANCE,      # 임베딩 거리를 기반으로 평가
    distance_metric=EmbeddingDistance.COSINE,        # 코사인 유사도 사용
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0), # OpenAI LLM 사용
    )


# 의미가 다른 문장 비교
result1 = embedding_evaluator.evaluate_strings(prediction="나는 학교에 갈 것이다", reference="나는 집에 있을 것이다")
print("의미가 다른 문장 비교 결과:", result1)

# 의미가 비슷한 문장 비교
result2 = embedding_evaluator.evaluate_strings(prediction="나는 학교에 갈 것이다", reference="나는 학교로 향할 것이다")
print("의미가 비슷한 문장 비교 결과:", result2)

의미가 다른 문장 비교 결과: {'score': 0.101075617379074}
의미가 비슷한 문장 비교 결과: {'score': 0.02741902364464399}


In [28]:
# test 데이터셋에 대한 평가
df_qa_test.head()

,context,source,doc_id,question,answer
0,"['.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2...",['data/리비안_KR.txt'],['0'],리비안의 초기 모델은 무엇인가요?,리비안의 초기 모델은 스포츠카 R1입니다.
1,"['.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2...",['data/리비안_KR.txt'],['0'],R1의 좌석 구성은 어떻게 되나요?,R1은 2+2 좌석 구성입니다.
2,"['.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2...",['data/리비안_KR.txt'],['0'],R1은 어떤 구조를 특징으로 하나요?,R1은 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 합니다.
3,"['테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전...",['data/테슬라_KR.txt'],['1'],테슬라는 어디에 본사를 두고 있나요?,테슬라는 텍사스주 오스틴에 본사를 두고 있습니다.
4,"['테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전...",['data/테슬라_KR.txt'],['1'],테슬라는 언제 설립되었나요?,테슬라는 2003년에 설립되었습니다.


In [29]:
# 첫번째 샘플에 대해 평가 - ground truth와 비교

question = df_qa_test.iloc[0]['question']
ground_truth = df_qa_test.iloc[0]['answer']

print("Question:", question)
print("Ground Truth:", ground_truth)

# OpenAI LLM을 사용하여 예측 생성
openai_prediction = openai_rag_chain.invoke(question)
print("Prediction:", openai_prediction)

distance_score = embedding_evaluator.evaluate_strings(prediction=openai_prediction, reference=ground_truth)
print("Distance Score:", distance_score)

Question: 리비안의 초기 모델은 무엇인가요?
Ground Truth: 리비안의 초기 모델은 스포츠카 R1입니다.
Prediction: 리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)입니다.
Distance Score: {'score': 0.03528604311768424}


In [30]:
### 다른 모델들에 대해 평가
# Anthropiic LLM을 사용하여 예측 생성
anthropic_prediction = anthropic_rag_chain.invoke(question)
print("Prediction:", anthropic_prediction)

distance_score = embedding_evaluator.evaluate_strings(prediction=anthropic_prediction, reference=ground_truth)
print("Distance Score:", distance_score)

Prediction: 리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페입니다.
Distance Score: {'score': 0.05677878923512858}


In [31]:
# Google Generative AI LLM을 사용하여 예측 생성
google_genai_prediction = google_genai_rag_chain.invoke(question)
print("Prediction:", google_genai_prediction)

distance_score = embedding_evaluator.evaluate_strings(prediction=google_genai_prediction, reference=ground_truth)
print("Distance Score:", distance_score)

Prediction: 리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)입니다. 

Distance Score: {'score': 0.04182225632635739}


In [32]:
# Ollama LLM을 사용하여 예측 생성
ollama_prediction = ollama_rag_chain.invoke(question)
print("Prediction:", ollama_prediction)

distance_score = embedding_evaluator.evaluate_strings(prediction=ollama_prediction, reference=ground_truth)
print("Distance Score:", distance_score)

Prediction: 스포츠카 R1(원래 이름은 Avera)입니다.
Distance Score: {'score': 0.09345460934189409}


In [33]:
# Groq LLM을 사용하여 예측 생성
groq_prediction = groq_rag_chain.invoke(question)
print("Prediction:", groq_prediction)

distance_score = embedding_evaluator.evaluate_strings(prediction=groq_prediction, reference=ground_truth)
print("Distance Score:", distance_score)

Prediction: 리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페입니다.
Distance Score: {'score': 0.05677878923512858}


In [34]:
# 모든 모델에 대해 평가 결과를 비교

def embedding_evaluate_all_models(question, ground_truth):
    results = {}
    for model_name, rag_chain in {
        "OpenAI": openai_rag_chain,
        "Anthropic": anthropic_rag_chain,
        "Google": google_genai_rag_chain,
        "Ollama": ollama_rag_chain,
        "Groq": groq_rag_chain,
    }.items():
        prediction = rag_chain.invoke(question)
        distance_score = embedding_evaluator.evaluate_strings(prediction=prediction, reference=ground_truth)
        results[model_name] = f"{distance_score['score']:.3f}"
    return results

question = df_qa_test.iloc[0]['question']
ground_truth = df_qa_test.iloc[0]['answer']

results = embedding_evaluate_all_models(question, ground_truth)
pprint(results)

{'Anthropic': '0.057',
 'Google': '0.053',
 'Groq': '0.057',
 'Ollama': '0.254',
 'OpenAI': '0.035'}


In [35]:
# 전체 데이터셋에 대해 A/B 테스트 - 여기서는 3개만 평가 (데이터프레임으로 정리)

def embedding_evaluate_qa_dataset(df_qa_test):

    results = []

    for i in range(3):
        question = df_qa_test.iloc[i]['question']
        ground_truth = df_qa_test.iloc[i]['answer']
        result = embedding_evaluate_all_models(question, ground_truth)
        results.append(result)

    return pd.DataFrame(results)

df_result_embedding = embedding_evaluate_qa_dataset(df_qa_test.iloc[:3])
df_result_embedding 

,OpenAI,Anthropic,Google,Ollama,Groq
0,0.035,0.057,0.055,0.119,0.057
1,0.004,0.092,0.057,0.067,0.012
2,-0.000,0.016,0.025,0.128,0.345


In [36]:
# Embedding Distance가 낮을 수록 좋은 결과
df_result_embedding.astype(float).mean().sort_values(ascending=True)

OpenAI       0.013000
Google       0.045667
Anthropic    0.055000
Ollama       0.104667
Groq         0.138000
dtype: float64

`(2) Cross-Encoder 활용`
- 두 문장을 동시에 인코딩하여 직접적으로 유사성을 평가 (개별 문장을 따로 인코딩하는 일반적인 Bi-Encoder 임베딩 방식과 다름)
- 크로스 인코더는 두 문장의 상호작용을 직접 모델링하므로, 일반적으로 더 정확한 유사성 점수를 제공
- 이 방법은 의미적 유사성을 더 정확하게 측정할 수 있지만, 계산 비용이 더 높다는 점을 유의해야 함

In [37]:
from sentence_transformers import CrossEncoder

def calculate_cross_encoder_similarity(
        query: str, 
        prediction: str, 
        model_name: str = "BAAI/bge-reranker-v2-m3",
        ) -> float:
    """
    주어진 query와 prediction 사이의 의미적 유사성을 계산합니다.

    Args:
    query (str): 기준이 되는 쿼리 문장
    prediction (str): 유사성을 비교할 예측 문장
    model_name (str): 사용할 크로스 인코더 모델 이름 (기본값: "BAAI/bge-reranker-v2-m3")

    Returns:
    float: 두 문장 간의 유사성 점수
    """
    # 크로스 인코더 모델을 불러옵니다.
    cross_encoder_model = CrossEncoder(model_name)

    # 크로스 인코더를 사용하여 유사성 점수를 계산합니다.
    sentence_pairs = [[query, prediction]]
    similarity_scores = cross_encoder_model.predict(sentence_pairs)

    return similarity_scores[0]

In [38]:
# 첫 번째 샘플에 대해 유사성 점수 계산 - 점수가 높을수록 더 유사함

print("Question:", question)
print("Ground Truth:", ground_truth)
print("Prediction:", openai_prediction)

similarity = calculate_cross_encoder_similarity(ground_truth, openai_prediction)
print(f"유사성 점수: {similarity:.4f}")

Question: 리비안의 초기 모델은 무엇인가요?
Ground Truth: 리비안의 초기 모델은 스포츠카 R1입니다.
Prediction: 리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)입니다.
유사성 점수: 0.9999


In [39]:
# 모든 모델에 대해 평가 결과를 비교

def cross_encoder_evaluate_all_models(question, ground_truth):
    results = {}
    for model_name, rag_chain in {
        "OpenAI": openai_rag_chain,
        "Anthropic": anthropic_rag_chain,
        "Google": google_genai_rag_chain,
        "Ollama": ollama_rag_chain,
        "Groq": groq_rag_chain,
    }.items():
        prediction = rag_chain.invoke(question)
        print(model_name)
        print(prediction)
        print()
        similarity = calculate_cross_encoder_similarity(ground_truth, prediction)
        results[model_name] = f"{similarity:.3f}"
    return results


# 전체 데이터셋에 대해 A/B 테스트 - 여기서는 3개만 평가 (데이터프레임으로 정리)

def cross_encoder_evaluate_qa_dataset(df_qa_test):

    results = []

    for i in range(3):
        question = df_qa_test.iloc[i]['question']
        ground_truth = df_qa_test.iloc[i]['answer']
        result = cross_encoder_evaluate_all_models(question, ground_truth)
        results.append(result)

    return pd.DataFrame(results)


df_result_cross_encoder = cross_encoder_evaluate_qa_dataset(df_qa_test.iloc[:3])
df_result_cross_encoder

OpenAI
리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)입니다.

Anthropic
리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페입니다.

Google
리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)입니다. 


Ollama
스포츠카 R1(원래 이름은 Avera)입니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다.

Groq


OpenAI
R1의 좌석 구성은 2+2 좌석입니다.

Anthropic
문맥에 따르면 리비안의 초기 모델인 R1은 2+2 좌석의 미드 엔진 하이브리드 쿠페라고 되어 있습니다. 따라서 R1의 좌석 구성은 2+2 좌석이라고 답변할 수 있습니다.

Google
R1은 2+2 좌석의 미드 엔진 하이브리드 쿠페입니다. 


Ollama
2+2 좌석입니다.

Groq
R1의 좌석 구성은 2+2 좌석입니다.

OpenAI
R1은 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 합니다.

Anthropic
문맥에 따르면, R1은 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 합니다.

Google
R1은 **모듈식 캡슐 구조**를 특징으로 합니다. 이 구조는 쉽게 교체 가능한 본체 패널을 갖추고 있습니다. 


Ollama
모듈식 캡슐 구조입니다.

Groq
R1은 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 합니다.



,OpenAI,Anthropic,Google,Ollama,Groq
0,1.000,1.000,1.000,0.474,0.000
1,1.000,0.999,0.974,0.998,1.000
2,1.000,1.000,1.000,0.996,1.000


In [40]:
# Cross Encoder Similarity가 높을 수록 좋은 결과
df_result_cross_encoder.astype(float).mean().sort_values(ascending=False)

OpenAI       1.000000
Anthropic    0.999667
Google       0.991333
Ollama       0.822667
Groq         0.666667
dtype: float64

`(3) Rouge Metric`
- ROUGE 메트릭은 두 문장의 유사도를 평가하는 데 널리 사용되는 방법으로, 특히 텍스트 요약과 기계 번역 분야에서 유용함
- 단어 중첩을 기반으로 하여 계산이 빠르고 해석이 쉽다는 장점이 있지만, 깊은 의미적 유사성을 포착하는 데는 한계가 있음
- 사용 목적과 컨텍스트에 따라 적절히 선택하거나 다른 메트릭과 조합하여 사용

In [41]:
from korouge_score import rouge_scorer
from typing import List, Dict

def calculate_rouge_similarity(
        query: str, 
        prediction: str, 
        rouge_types: List[str] = ['rouge1', 'rouge2', 'rougeL'],
        ) -> Dict[str, float]:
    
    """
    주어진 쿼리 문장과 예측 문장 사이의 선택된 ROUGE 점수를 계산합니다.

    Args:
    query (str): 기준이 되는 쿼리 문장
    prediction (str): 유사성을 비교할 예측 문장
    rouge_types (List[str]): 계산할 ROUGE 메트릭 리스트 (기본값: ['rouge1', 'rouge2', 'rougeL'])

    Returns:
    Dict[str, float]: 선택된 ROUGE 메트릭의 F1 점수를 포함하는 딕셔너리
    """
    # 입력된 ROUGE 유형의 유효성을 검사합니다.
    valid_rouge_types = set(['rouge1', 'rouge2', 'rougeL'])
    rouge_types = [rt for rt in rouge_types if rt in valid_rouge_types]
    
    if not rouge_types:
        raise ValueError("유효한 ROUGE 유형이 제공되지 않았습니다.")

    # ROUGE scorer 객체를 초기화합니다.
    scorer = rouge_scorer.RougeScorer(rouge_types, use_stemmer=True)

    # ROUGE 점수를 계산합니다.
    scores = scorer.score(query, prediction)

    # 결과를 정리합니다.
    result = {rouge_type: scores[rouge_type].fmeasure for rouge_type in rouge_types}

    return result

In [42]:
# 첫 번째 샘플에 대해 유사성 점수 계산 - 점수가 높을수록 더 유사함

print("Question:", question)
print("Ground Truth:", ground_truth)
print("Prediction:", openai_prediction)

rouge_scores = calculate_rouge_similarity(ground_truth, openai_prediction, ['rouge1'])
print(f"Rouge 점수: {rouge_scores['rouge1']:.4f}")

Question: 리비안의 초기 모델은 무엇인가요?
Ground Truth: 리비안의 초기 모델은 스포츠카 R1입니다.
Prediction: 리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)입니다.
Rouge 점수: 0.6667


In [46]:
# 모든 모델에 대해 평가 결과를 비교

def rouge_evaluate_all_models(question, ground_truth):
    results = {}
    for model_name, rag_chain in {
        "OpenAI": openai_rag_chain,
        "Anthropic": anthropic_rag_chain,
        "Google": google_genai_rag_chain,
        "Ollama": ollama_rag_chain,
        "Groq": groq_rag_chain,
    }.items():
        prediction = rag_chain.invoke(question)
        rouge_scores = calculate_rouge_similarity(ground_truth, prediction, ['rouge2'])
        results[model_name] = f"{rouge_scores['rouge2']:.3f}"
    return results

# 전체 데이터셋에 대해 A/B 테스트 - 여기서는 3개만 평가 (데이터프레임으로 정리)

def rouge_evaluate_qa_dataset(df_qa_test):
    
        results = []
    
        for i in range(3):
            question = df_qa_test.iloc[i]['question']
            ground_truth = df_qa_test.iloc[i]['answer']
            result = rouge_evaluate_all_models(question, ground_truth)
            results.append(result)
    
        return pd.DataFrame(results)    

df_result_rouge = rouge_evaluate_qa_dataset(df_qa_test.iloc[:3])
df_result_rouge

,OpenAI,Anthropic,Google,Ollama,Groq
0,0.600,0.375,0.545,0.000,0.375
1,1.000,0.080,0.000,0.000,0.000
2,1.000,0.917,0.417,0.609,1.000


In [47]:
# ROUGE 점수가 높을 수록 좋은 결과
df_result_rouge.astype(float).mean().sort_values(ascending=False)

OpenAI       0.866667
Groq         0.458333
Anthropic    0.457333
Google       0.320667
Ollama       0.203000
dtype: float64

## 4-2 LLM-as-judge
- LLM은 인간과 유사한 판단을 제공할 수 있어, 단순한 단어 중첩 기반 메트릭보다 더 깊은 의미적 관련성을 포착할 수 있음   
- Rouge와 같은 전통적인 메트릑보다 더 정교한 평가를 제동할 수 있지만, LLM의 판단에 의존하기 때문에 완전한 객관성을 보장하기는 어려움   
- 또한 API 사용에 따른 비용과 처리 시간이 필요하다는 점을 고려해야 함   
- 따라서 여러 평가 방법을 조합하여 사용 필요 (예를 들어, ROUGE 점수로 빠른 초기 스크리닝을 하고, 중요한 케이스에 대해서만 이 LLM 기반 평가를 수행하는 방법을 고려)

(1) QA Evaluation - 사용자 질문에 대한 정확성, 관련성을 평가 (Y/N, 0/1)   
    1. QA 평가기 : 사용자 질문에 대한 응답의 정확성을 직접적으로 평가   
    2. CONTEXT_QA 평가기 : 더 넓은 맥락을 고려하여 응답의 정확성을 평가   
    3. COT_QA 평가기 : cot 추론을 통해 더 심층적인 평가를 수행

In [6]:
# 2번째 샘플에 대해 예측 수행
question = df_qa_test.iloc[1]['question']
context = df_qa_test.iloc[1]['context']
ground_truth = df_qa_test.iloc[1]['answer']
#groq_prediction = groq_rag_chain.invoke(question)
groq_prediction = "dddd"
print("Question:", question)
print("Context:", context)
print("Ground Truth:", ground_truth)
print("Prediction:", groq_prediction)

Question: R1의 좌석 구성은 어떻게 되나요?
Context: ['.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다\n\n(참고: 이 문서는 리비안에 대한 정보를 담고 있습니다.)']
Ground Truth: R1은 2+2 좌석 구성입니다.
Prediction: dddd


In [7]:
from langchain.evaluation import load_evaluator, EvaluatorType
from langchain_openai import ChatOpenAI

# QA Evaluator 초기화
qa_evaluator = load_evaluator(
    evaluator="qa",
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
)

# 2번째 샘플에 대해 평가 수행
qa_eval_result = qa_evaluator.evaluate_strings(
    input=question,
    prediction=groq_prediction,
    reference=ground_truth
)

# 결과 출력
qa_eval_result

{'reasoning': 'INCORRECT', 'value': 'INCORRECT', 'score': 0}

In [ ]:
from langchain.evaluation import load_evaluator, EvaluatorType
from langchain_openai import ChatOpenAI

# context_qa Evaluator 초기화
qa_evaluator = load_evaluator(
    evaluator="context_qa",
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
)

# 2번째 샘플에 대해 평가 수행
context_qa_eval_result = qa_evaluator.evaluate_strings(
    input=question,
    prediction=groq_prediction,
    reference=context
)

# 결과 출력
context_qa_eval_result

In [2]:
from langchain.evaluation import load_evaluator, EvaluatorType
from langchain_openai import ChatOpenAI

# cot qa 초기화
qa_evaluator = load_evaluator(
    evaluator="cot_qa",
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
)

# 2번째 샘플에 대해 평가 수행
cot_qa_eval_result = qa_evaluator.evaluate_strings(
    input=question,
    prediction=groq_prediction,
    reference=context
)

# 결과 출력
cot_qa_eval_result

NameError: name 'question' is not defined

In [ ]:
print(cot_qa_eval_result["reasoning"])

In [10]:
# COT QA Evaluator 초기화 (사용자 정의 프롬프트 사용)
from langchain_core.prompts.prompt import PromptTemplate

CUSTOM_COT_QA_PROMPT = """
You are an expert evaluating the performance of a RAG (Retrieval-Augmented Generation) system. Your task is to assess the quality of the system's answer based on the given question, retrieved context, and the generated answer. Grade the answer as CORRECT, PARTIALLY CORRECT, or INCORRECT, and provide a detailed explanation. Please describe your evaluation process step by step to clearly show how you reached your conclusion.

Use the following criteria for evaluation:
1. Factual Accuracy: Does the answer align with the information in the retrieved context?
2. Relevance: Does the answer appropriately address the question?
3. Completeness: Does the answer cover all aspects of the question?
4. Conciseness: Does the answer convey the key information without unnecessary details?

Grading Criteria:
- CORRECT: Meets all criteria with no errors
- PARTIALLY CORRECT: Meets some criteria but has minor errors or omissions
- INCORRECT: Contains major factual errors or is irrelevant to the question

Evaluation Format:
QUESTION: [The question content]
CONTEXT: [The retrieved context]
SYSTEM ANSWER: [The answer generated by the RAG system]
EVALUATION:
1. Factual Accuracy: [Assessment and explanation]
2. Relevance: [Assessment and explanation]
3. Completeness: [Assessment and explanation]
4. Conciseness: [Assessment and explanation]
Overall Assessment: [General evaluation summary]
GRADE: [CORRECT / PARTIALLY CORRECT / INCORRECT]

Now, please evaluate the RAG system's answer based on the provided information. Prefer to use in Korean language.

QUESTION: {query}
CONTEXT: {context}
SYSTEM ANSWER: {result}
EVALUATION:
"""

custom_qa_prompt = PromptTemplate.from_template(CUSTOM_COT_QA_PROMPT)

custom_cot_qa_evaluator = load_evaluator(
    evaluator="cot_qa",  
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0), # OpenAI LLM 사용
    prompt=custom_qa_prompt                               # 사용자 지정 프롬프트 사용
    )

# 2번째 샘플에 대해 평가 수행
custom_cot_qa_eval_result = custom_cot_qa_evaluator.evaluate_strings(
    input=question,               # 평가에 고려할 내용: 질문
    prediction="ddd",   # 평가 대상: LLM 모델의 예측
    reference=context,            # 평가 기준: 문맥
)

# 결과 출력
custom_cot_qa_eval_result

{'reasoning': 'QUESTION: R1의 좌석 구성은 어떻게 되나요?  \nCONTEXT: [\'.\\n\\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다\\n\\n(참고: 이 문서는 리비안에 대한 정보를 담고 있습니다.)\']  \nSYSTEM ANSWER: ddd  \n\nEVALUATION:  \n1. Factual Accuracy:  \n   - 평가: 시스템의 답변 "ddd"는 질문에 대한 정보가 전혀 포함되어 있지 않으며, 제공된 문맥에서 R1의 좌석 구성에 대한 정보(2+2 좌석)가 명확히 언급되어 있습니다. 따라서, 사실적으로 정확하지 않습니다.  \n   - 설명: 질문은 R1의 좌석 구성에 대한 것이며, 문맥에서는 "2+2 좌석"이라는 구체적인 정보가 제공되었으나, 시스템의 답변은 이와 관련이 없습니다.  \n\n2. Relevance:  \n   - 평가: 시스템의 답변은 질문과 전혀 관련이 없습니다.  \n   - 설명: 질문은 R1의 좌석 구성에 대한 것이지만, "ddd"라는 답변은 의미가 없고 질문에 대한 적절한 답변이 아닙니다.  \n\n3. Completeness:  \n   - 평가: 시스템의 답변은 질문에 대한 어떤 정보도 제공하지 않으므로, 완전성이 결여되어 있습니다.  \n   - 설명: 질문에 대한 답변이 전혀 없기 때문에, 질문의 모든 측면을 다루지 못하고 있습니다.  \n\n4. Conciseness:  \n   - 평가: 시스템의 답변은 간결함을 갖추고 있지 않으며, 의미가 없는 문자열로 구성되어 있습니다.  \n   - 설명: "ddd"는 질문에 대한 유의미한 정보를 전달하지 않기 때문에, 간결함의 기준을 충족하지 않습니다.  \n\nOverall Assessment:

(2) Criteria Evaluation (No lables) - 참조 레이블(ground truth)이 없는 상황에서 모델 출력의 품질을 평가   
   1. "criteria" 평가기 :   
       목적 : 주어진 기준에 따라 예측이 기준을 만족하는지 평가   
       출력 : 이진 점수 (예 YES/NO or 1/0)   
   2. "score_string" 평가기:   
       목적 : 주어진 기준에 따라 예측의 품질을 수치로 평가   
       출력 : 수치 점수 (기본적으로 1-10 척도)   

In [11]:
from langchain.evaluation import load_evaluator

#간결성 평가 - criteria 평가자 사용
conciseness_evaluator = load_evaluator(
    evaluator="criteria",
    criteria="conciseness",
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
)

# 2번째 샘플에 대해 평가 수행
conciseness_result = conciseness_evaluator.evaluate_strings(
    input=question,                   # 질문 
    prediction="sss",       # 평가 대상: LLM 모델의 예측
)

# 결과 출력
conciseness_result

{'reasoning': 'To assess whether the submission meets the criteria of conciseness, I will evaluate the submission step by step.\n\n1. **Understanding the Input**: The input question is asking about the seating arrangement of R1. This implies that the expected response should provide specific information regarding the seating configuration.\n\n2. **Analyzing the Submission**: The submission provided is "sss". This response does not contain any relevant information about the seating arrangement of R1. It appears to be a random string of characters rather than a meaningful answer.\n\n3. **Evaluating Conciseness**: Conciseness means that the response should be brief but still convey the necessary information. In this case, the submission fails to provide any information at all, which means it is not concise; it is irrelevant.\n\n4. **Conclusion**: Since the submission does not address the question and fails to provide any relevant information, it does not meet the criterion of being concis

In [15]:
# criteria 직접 지정
criteria_evaluator = load_evaluator(
    evaluator="criteria", 
    criteria={
        "relevance": "Does the answer appropriately address the question?",
        "conciseness": "Does the answer convey the key information without unnecessary details?",
        "hello":"is hello?"
        },
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0),
    )

# 2번째 샘플에 대해 평가 수행
criteria_result = criteria_evaluator.evaluate_strings(
    input=question,              # 질문 
    prediction="sss",      # 평가 대상: LLM 모델의 예측
)

# 결과 출력
criteria_result

{'reasoning': 'To assess the submission based on the provided criteria, I will evaluate each criterion step by step.\n\n1. **Relevance**: The question asks about the seating arrangement of R1. The submission "sss" does not provide any information related to the seating configuration. Therefore, it does not appropriately address the question.\n\n2. **Conciseness**: While the submission is short and does not contain unnecessary details, it fails to convey any relevant information regarding the question. Conciseness is not sufficient if the answer is irrelevant.\n\n3. **Hello**: The criterion "is hello?" seems to be a nonsensical or unrelated criterion in this context. The submission does not include the word "hello," but since this criterion does not pertain to the relevance or conciseness of the answer, it does not impact the overall assessment.\n\nBased on the evaluation:\n- The submission fails the relevance criterion.\n- The submission does not meet the conciseness criterion in a mea

In [16]:
print(criteria_result['reasoning'])

To assess the submission based on the provided criteria, I will evaluate each criterion step by step.

1. **Relevance**: The question asks about the seating arrangement of R1. The submission "sss" does not provide any information related to the seating configuration. Therefore, it does not appropriately address the question.

2. **Conciseness**: While the submission is short and does not contain unnecessary details, it fails to convey any relevant information regarding the question. Conciseness is not sufficient if the answer is irrelevant.

3. **Hello**: The criterion "is hello?" seems to be a nonsensical or unrelated criterion in this context. The submission does not include the word "hello," but since this criterion does not pertain to the relevance or conciseness of the answer, it does not impact the overall assessment.

Based on the evaluation:
- The submission fails the relevance criterion.
- The submission does not meet the conciseness criterion in a meaningful way since it lack

In [ ]:
# score_string 평가자 사용
score_string_evaluator = load_evaluator(
    evaluator="score_string", 
    criteria={
        "relevance": "How relevant is the answer to the question on a scale of 1-10?",
    },
    normalize_by=10,
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.0),
    )

# 2번째 샘플에 대해 평가 수행
score_string_result = score_string_evaluator.evaluate_strings(
    input=question,                     # 질문 
    prediction=groq_prediction,         # 평가 대상: LLM 모델의 예측
)

# 결과 출력
score_string_result